In [1]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
# Read the stock data file
stocks_df = pd.read_csv('/content/drive/MyDrive/Python+for+Financial+Analysis+Package/Part 2. Financial Analysis in Python/stock.csv')
stocks_df.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.13,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.35,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.25,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.73,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.80,194.449997,180.520004,26.760000,318.590851,1314.500000


In [3]:
# Sort the data based on Date
stocks_df = stocks_df.sort_values(by = ['Date'])
stocks_df.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.13,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.35,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.25,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.73,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.80,194.449997,180.520004,26.760000,318.590851,1314.500000


In [4]:
# Function to normalize the prices based on the initial price
def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

In [5]:
# Function to plot interactive plot
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [6]:
# Plot interactive chart
interactive_plot(stocks_df, 'Prices')

In [7]:
# Plot normalized interactive chart
interactive_plot(normalize(stocks_df), 'Normalized Prices')

###### PERFORM RANDOM ASSET ALLOCATION AND CALCULATE PORTFOLIO DAILY RETURN

In [8]:
# Let's create random portfolio weights
# Portfolio weights must sum to 1 

# Set random seed
# np.random.seed(101)
np.random.seed()

# Create random weights for the stocks and normalize them
weights = np.array(np.random.random(9))

# Ensure that the sum of all weights are = 1
weights = weights / np.sum(weights) 
print(weights)



[0.15023215 0.00333599 0.19461882 0.19076702 0.01253728 0.16244335
 0.10301531 0.13318755 0.04986252]


In [9]:
# Normalize the stock avalues 
df_portfolio = normalize(stocks_df)
df_portfolio.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2012-01-13,0.996251,0.987949,0.998340,1.018137,1.014153,0.992301,0.806726,0.992615,0.995052
2,2012-01-17,1.007855,0.996424,1.004316,1.009893,1.032570,0.996954,0.941593,0.998317,0.998587
3,2012-01-18,1.018320,0.994040,1.006972,1.049464,1.076792,1.002880,0.949027,1.005193,1.009680
4,2012-01-19,1.015093,1.000662,1.009960,1.055235,1.105269,0.999834,0.947257,1.015771,1.014666


In [10]:
df_portfolio.columns[1:]

Index(['AAPL', 'BA', 'T', 'MGM', 'AMZN', 'IBM', 'TSLA', 'GOOG', 'sp500'], dtype='object')

In [11]:
# Note that enumerate returns the value and a counter as well
for counter, stock in enumerate(df_portfolio.columns[1:]):
  df_portfolio[stock] = df_portfolio[stock] * weights[counter]
  df_portfolio[stock] = df_portfolio[stock] * 1000000
df_portfolio.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,150232.154740,3335.994872,194618.819442,190767.016425,12537.278677,162443.353030,103015.309730,133187.553024,49862.520061
1,2012-01-13,149668.865610,3295.791341,194295.740581,194226.929336,12714.723615,161192.751565,83105.097761,132203.941177,49615.804161
2,2012-01-17,151412.226400,3324.066227,195458.801217,192654.241649,12945.615786,161948.507669,96998.486330,132963.334034,49792.086856
3,2012-01-18,152984.463211,3316.113916,195975.717055,200203.142546,13500.040881,162911.207874,97764.260207,133879.259997,50345.173821
4,2012-01-19,152499.598627,3338.203670,196557.247372,201304.023927,13857.067573,162416.362512,97581.935871,135288.048189,50593.811363


In [12]:
# Let's create an additional column that contains the sum of all $ values in the portfolio
df_portfolio['portfolio daily worth in $'] = df_portfolio[df_portfolio != 'Date'].sum(axis = 1)
df_portfolio.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $
0,2012-01-12,150232.154740,3335.994872,194618.819442,190767.016425,12537.278677,162443.353030,103015.309730,133187.553024,49862.520061,1.000000e+06
1,2012-01-13,149668.865610,3295.791341,194295.740581,194226.929336,12714.723615,161192.751565,83105.097761,132203.941177,49615.804161,9.803196e+05
2,2012-01-17,151412.226400,3324.066227,195458.801217,192654.241649,12945.615786,161948.507669,96998.486330,132963.334034,49792.086856,9.974974e+05
3,2012-01-18,152984.463211,3316.113916,195975.717055,200203.142546,13500.040881,162911.207874,97764.260207,133879.259997,50345.173821,1.010879e+06
4,2012-01-19,152499.598627,3338.203670,196557.247372,201304.023927,13857.067573,162416.362512,97581.935871,135288.048189,50593.811363,1.013436e+06


In [13]:
# Let's calculate the portfolio daily return 
# Define a new column in the dataframe and set it to zeros
df_portfolio['portfolio daily % return'] = 0.0000

for i in range(1, len(stocks_df)):
  # Calculate the percentage of change from the previous day
  df_portfolio['portfolio daily % return'][i] = ( (df_portfolio['portfolio daily worth in $'][i] - df_portfolio['portfolio daily worth in $'][i-1]) / df_portfolio['portfolio daily worth in $'][i-1]) * 100 

df_portfolio.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,150232.154740,3335.994872,194618.819442,190767.016425,12537.278677,162443.353030,103015.309730,133187.553024,49862.520061,1.000000e+06,0.000000
1,2012-01-13,149668.865610,3295.791341,194295.740581,194226.929336,12714.723615,161192.751565,83105.097761,132203.941177,49615.804161,9.803196e+05,-1.968035
2,2012-01-17,151412.226400,3324.066227,195458.801217,192654.241649,12945.615786,161948.507669,96998.486330,132963.334034,49792.086856,9.974974e+05,1.752257
3,2012-01-18,152984.463211,3316.113916,195975.717055,200203.142546,13500.040881,162911.207874,97764.260207,133879.259997,50345.173821,1.010879e+06,1.341559
4,2012-01-19,152499.598627,3338.203670,196557.247372,201304.023927,13857.067573,162416.362512,97581.935871,135288.048189,50593.811363,1.013436e+06,0.252940


###### PORTFOLIO ALLOCATION - DAILY RETURN/WORTH CALCULATION (FUNCTION)

In [14]:
# Lets assume we have $1,000,000 to be invested and we will allocate this fund based on the weights of the stocks
# We will create a function that takes in the stock prices along with the weights and retun:
# (1) Daily value of each individual securuty in $ over the specified time period
# (2) Overall daily worth of the entire portfolio 
# (3) Daily return 

def portfolio_allocation(df, weights):

  df_portfolio = df.copy()
  
  # Normalize the stock avalues 
  df_portfolio = normalize(df_portfolio)
  
  for counter, stock in enumerate(df_portfolio.columns[1:]):
    df_portfolio[stock] = df_portfolio[stock] * weights[counter]
    df_portfolio[stock] = df_portfolio[stock] * 1000000

  df_portfolio['portfolio daily worth in $'] = df_portfolio[df_portfolio != 'Date'].sum(axis = 1)
  
  df_portfolio['portfolio daily % return'] = 0.0000

  for i in range(1, len(stocks_df)):
    
    # Calculate the percentage of change from the previous day
    df_portfolio['portfolio daily % return'][i] = ( (df_portfolio['portfolio daily worth in $'][i] - df_portfolio['portfolio daily worth in $'][i-1]) / df_portfolio['portfolio daily worth in $'][i-1]) * 100 
  
  # set the value of first row to zero, as previous value is not available
  df_portfolio['portfolio daily % return'][0] = 0
  return df_portfolio

In [15]:
# Call the function
df_portfolio = portfolio_allocation(stocks_df, weights)
df_portfolio.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,150232.154740,3335.994872,194618.819442,190767.016425,12537.278677,162443.353030,103015.309730,133187.553024,49862.520061,1.000000e+06,0.000000
1,2012-01-13,149668.865610,3295.791341,194295.740581,194226.929336,12714.723615,161192.751565,83105.097761,132203.941177,49615.804161,9.803196e+05,-1.968035
2,2012-01-17,151412.226400,3324.066227,195458.801217,192654.241649,12945.615786,161948.507669,96998.486330,132963.334034,49792.086856,9.974974e+05,1.752257
3,2012-01-18,152984.463211,3316.113916,195975.717055,200203.142546,13500.040881,162911.207874,97764.260207,133879.259997,50345.173821,1.010879e+06,1.341559
4,2012-01-19,152499.598627,3338.203670,196557.247372,201304.023927,13857.067573,162416.362512,97581.935871,135288.048189,50593.811363,1.013436e+06,0.252940


In [16]:
# Plot the portfolio daily return
fig = px.line(x = df_portfolio.Date, y = df_portfolio['portfolio daily % return'], title = 'Portfolio Daily % Return')
fig.show()

In [17]:
# Plot all stocks (normalized)
interactive_plot(df_portfolio.drop(['portfolio daily worth in $', 'portfolio daily % return'], axis = 1), 'Portfolio individual stocks worth in $ over time')

In [18]:
# Print out a histogram of daily returns
fig = px.histogram(df_portfolio, x = 'portfolio daily % return')
fig.show()

In [19]:
# Plot the portfolio overall daily worth vs. time
fig = px.line(x = df_portfolio.Date, y = df_portfolio['portfolio daily worth in $'], title= 'Portfolio Overall Value in $')
fig.show()

###### CALCULATING PORTFOLIO STATISTICAL METRICS (CUMMULATIVE RETURN, AVERAGE DAILY RETURN, AND SHARPE RATIO)

In [20]:
# sharpe ratio is a calculation of average return earned in excess of risk free rate. 
# sharpe ratio is calculated to underdstand the return of an investment when compared to risk. 
#As sharpe ratio increases, risk-adjusted return increases and security becomes desirable for investors
#cumulative return- amount stock has lost or gained over the period of time

In [21]:
df_portfolio

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,1.502322e+05,3335.994872,194618.819442,190767.016425,12537.278677,162443.353030,1.030153e+05,133187.553024,49862.520061,1.000000e+06,0.000000
1,2012-01-13,1.496689e+05,3295.791341,194295.740581,194226.929336,12714.723615,161192.751565,8.310510e+04,132203.941177,49615.804161,9.803196e+05,-1.968035
2,2012-01-17,1.514122e+05,3324.066227,195458.801217,192654.241649,12945.615786,161948.507669,9.699849e+04,132963.334034,49792.086856,9.974974e+05,1.752257
3,2012-01-18,1.529845e+05,3316.113916,195975.717055,200203.142546,13500.040881,162911.207874,9.776426e+04,133879.259997,50345.173821,1.010879e+06,1.341559
4,2012-01-19,1.524996e+05,3338.203670,196557.247372,201304.023927,13857.067573,162416.362512,9.758194e+04,135288.048189,50593.811363,1.013436e+06,0.252940
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,1.098692e+06,7699.604920,192874.222027,262953.365528,228399.683063,112869.110007,5.415214e+06,625761.279826,128082.593115,8.072546e+06,0.061665
2155,2020-08-06,1.137025e+06,7607.711451,192809.607547,290318.131281,229822.800789,113471.923739,5.431842e+06,637010.125070,128905.869008,8.168813e+06,1.192522
2156,2020-08-07,1.109174e+06,7511.400429,193972.668183,299282.482550,225722.331666,112428.251980,5.297394e+06,634627.871926,128987.470225,8.009100e+06,-1.955159
2157,2020-08-10,1.125296e+06,7926.246026,195135.735280,340486.884221,224346.954514,114362.638732,5.172900e+06,635311.544614,129341.182048,7.945107e+06,-0.799007


In [22]:
# Cummulative return of the portfolio (Note that we now look for the last net worth of the portfolio compared to it's start value)
cummulative_return = ((df_portfolio['portfolio daily worth in $'][-1:] - df_portfolio['portfolio daily worth in $'][0])/ df_portfolio['portfolio daily worth in $'][0]) * 100
print('Cummulative return of the portfolio is {} %'.format(cummulative_return.values[0]))


Cummulative return of the portfolio is 673.5343423385114 %


In [23]:
# Calculate the portfolio standard deviation
print('Standard deviation of the portfolio is {}'.format(df_portfolio['portfolio daily % return'].std()))

Standard deviation of the portfolio is 1.7368968770300177


In [24]:
# Calculate the average daily return 
print('Average daily return of the portfolio is {} %'.format(df_portfolio['portfolio daily % return'].mean() ))

Average daily return of the portfolio is 0.10992723362335077 %


In [25]:
# Portfolio sharpe ratio
sharpe_ratio = df_portfolio['portfolio daily % return'].mean() / df_portfolio['portfolio daily % return'].std() * np.sqrt(252)
print('Sharpe ratio of the portfolio is {}'.format(sharpe_ratio))

Sharpe ratio of the portfolio is 1.004688739994749
